In [2]:
import pandas as pd
import s3fs
from smart_open import open
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import metrics
import torch
from transformers import *
import numpy as np
import ast
import time
import metrics

### Reply

In [ ]:
column_of_interest = ["text_ tokens", "engaging_user_id", "reply_timestamp"]
train_set = pd.read_csv('s3://recsys-challenge-2020/training.tsv', encoding="utf-8",  sep="\x01",
                     usecols= [0, 14, 20], names = column_of_interest)
column_of_interest = ["engaging_user_id"]
val_set = pd.read_csv('s3://recsys-challenge-2020/val.tsv', encoding="utf-8", sep="\x01",
                     usecols= [14], names=column_of_interest)
unique_val_set_users = val_set[['engaging_user_id']].drop_duplicates()
train_set_positive = train_set[train_set.reply_timestamp.notnull()]
warm_train_set_positive = pd.merge(train_set_positive, unique_val_set_users, how = 'inner', on = 'engaging_user_id')
user_tokens = warm_train_set_positive.groupby(['engaging_user_id'])['text_ tokens'].agg(list).reset_index()
user_tokens.to_csv('s3://recsys-challenge-2020/user_tokens_reply.csv', index = False)

### Retweet

In [3]:
column_of_interest = ["text_ tokens", "engaging_user_id", "retweet_timestamp"]
train_set = pd.read_csv('s3://recsys-challenge-2020/training.tsv', encoding="utf-8",  sep="\x01",
                     usecols= [0, 14, 21], names = column_of_interest)
column_of_interest = ["engaging_user_id"]
val_set = pd.read_csv('s3://recsys-challenge-2020/val.tsv', encoding="utf-8", sep="\x01",
                     usecols= [14], names=column_of_interest)
unique_val_set_users = val_set[['engaging_user_id']].drop_duplicates()
train_set_positive = train_set[train_set.retweet_timestamp.notnull()]
warm_train_set_positive = pd.merge(train_set_positive, unique_val_set_users, how = 'inner', on = 'engaging_user_id')
user_tokens = warm_train_set_positive.groupby(['engaging_user_id'])['text_ tokens'].agg(list).reset_index()
user_tokens.to_csv('s3://recsys-challenge-2020/user_tokens_retweet.csv', index = False)

### Retweet with comment

In [3]:
column_of_interest = ["text_ tokens", "engaging_user_id", "retweet_with_comment_timestamp"]
train_set = pd.read_csv('s3://recsys-challenge-2020/training.tsv', encoding="utf-8",  sep="\x01",
                     usecols= [0, 14, 22], names = column_of_interest)
column_of_interest = ["engaging_user_id"]
val_set = pd.read_csv('s3://recsys-challenge-2020/val.tsv', encoding="utf-8", sep="\x01",
                     usecols= [14], names=column_of_interest)
unique_val_set_users = val_set[['engaging_user_id']].drop_duplicates()
train_set_positive = train_set[train_set.retweet_with_comment_timestamp.notnull()]
warm_train_set_positive = pd.merge(train_set_positive, unique_val_set_users, how = 'inner', on = 'engaging_user_id')
user_tokens = warm_train_set_positive.groupby(['engaging_user_id'])['text_ tokens'].agg(list).reset_index()
user_tokens.to_csv('s3://recsys-challenge-2020/user_tokens_retweet_with_comment.csv', index = False)

### like

In [2]:
column_of_interest = ["text_ tokens", "engaging_user_id", "like_timestamp"]
train_set = pd.read_csv('s3://recsys-challenge-2020/training.tsv', encoding="utf-8",  sep="\x01",
                     usecols= [0, 14, 23], names = column_of_interest)
column_of_interest = ["engaging_user_id"]
val_set = pd.read_csv('s3://recsys-challenge-2020/val.tsv', encoding="utf-8", sep="\x01",
                     usecols= [14], names=column_of_interest)
unique_val_set_users = val_set[['engaging_user_id']].drop_duplicates()
train_set_positive = train_set[train_set.like_timestamp.notnull()]
warm_train_set_positive = pd.merge(train_set_positive, unique_val_set_users, how = 'inner', on = 'engaging_user_id')
user_tokens = warm_train_set_positive.groupby(['engaging_user_id'])['text_ tokens'].agg(list).reset_index()
user_tokens.to_csv('s3://recsys-challenge-2020/user_tokens_like.csv', index = False)

### Tokens exceed 512

In [3]:
column_of_interest = ["engaging_user_id"]
val_set = pd.read_csv('s3://recsys-challenge-2020/val.tsv', encoding="utf-8", sep="\x01",
                     usecols= [14], names=column_of_interest)

In [4]:
pd.set_option('display.max_colwidth', -1)
model = BertModel.from_pretrained('/dev/bert/')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [5]:
val_set.loc[1119671]

engaging_user_id    12E9952B6B5F58BA0A83B8116CCF19D1
Name: 1119671, dtype: object

In [ ]:
user_tokens = pd.read_csv('s3://recsys-challenge-2020/user_tokens_reply.csv')